In [9]:
from fetchData import metadata_price_action, extended_stock_stats, basic_stats, current_sp500_symbols, parse_value
import numpy as np
import pandas as pd
import math

In [2]:
symbol="IBM"
timeInterval = "1d"
timeRange = "10y"

In [3]:
sp500symbols = np.array(current_sp500_symbols())
sp500symbols

array(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP',
       'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS',
       'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T',
       'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC',
       'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA',
       'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW',
       'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT',
       'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF',
       'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS',
       'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH',
       'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT',


In [4]:
my_columns = ["Ticker", "Stock Price", "Market Cap", "52 Week Change", "Number of Shares to Buy"]
dataframe = pd.DataFrame(columns=my_columns)
for sym in sp500symbols[0:4]:
    if(sym.find(".") > -1):
        sym = sym.replace(".", "-")
    basic = basic_stats(sym)
    complexStats = extended_stock_stats(sym)
    marketCap = parse_value(basic["Market Cap"])
    previousClose = parse_value(basic["Previous Close"])
    fiftyTwoWeekChange = previousClose / (1+parse_value(complexStats["52-Week Change 3"]))
    dataframe = dataframe.append(pd.Series([sym,previousClose,round(marketCap),fiftyTwoWeekChange,"N/A"], index=my_columns), ignore_index=True)
    print(sym, round(marketCap), previousClose)
dataframe.to_csv("lastDataframe_bak.csv")
dataframe

STATS <Response [200]>
MMM 101377000000 178.7
STATS <Response [200]>
ABT 212194000000 121.5
STATS <Response [200]>
ABBV 190256000000 107.3
STATS <Response [200]>
ABMD 14680000000 320.4


,Ticker,Stock Price,Market Cap,52 Week Change,Number of Shares to Buy
0,MMM,178.7,101377000000,152.970382,N/A
1,ABT,121.5,212194000000,81.554571,N/A
2,ABBV,107.3,190256000000,88.663031,N/A
3,ABMD,320.4,14680000000,155.067273,N/A


In [5]:
dataframe.to_csv("lastDataframe.csv")

In [6]:
portfolio_size = input("How large is your portfolio?")

How large is your portfolio? 20


In [7]:
portfolio_size = float(portfolio_size)
portfolio_size

20.0

In [10]:
position_size = portfolio_size / len(dataframe.index)
print(position_size)
for i in range(0,len(dataframe.index)):
    dataframe.loc[i,"Number of Shares to Buy"] = math.floor(position_size/dataframe.loc[i,"Stock Price"])

dataframe.to_csv("lastDataframeWithSharesToBuy.csv")
dataframe

5.0


,Ticker,Stock Price,Market Cap,52 Week Change,Number of Shares to Buy
0,MMM,178.7,101377000000,152.970382,0
1,ABT,121.5,212194000000,81.554571,0
2,ABBV,107.3,190256000000,88.663031,0
3,ABMD,320.4,14680000000,155.067273,0


In [11]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'IBM',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': -252322200,
 'regularMarketTime': 1614373203,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 118.93,
 'chartPreviousClose': 162.28,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1614330000,
   'end': 1614349800,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1614349800,
   'end': 1614373200,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1614373200,
   'end': 1614387600,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '10y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [12]:
price = metadataAndPrice["price"]
price

,timestamp,high,low,volume,open,close
0,1298903400,162.990005,161.240005,4773000,162.360001,161.880005
1,1298989800,163.149994,159.880005,4382800,163.149994,159.970001
2,1299076200,161.110001,159.410004,3413000,159.429993,160.160004
3,1299162600,164.000000,161.270004,4057800,161.419998,163.479996
4,1299249000,164.309998,160.649994,4910700,163.399994,161.830002
...,...,...,...,...,...,...
2512,1614004200,121.129997,118.440002,5877300,118.500000,120.860001
2513,1614090600,121.349998,119.800003,4531300,120.919998,120.709999
2514,1614177000,123.489998,120.519997,5818100,120.800003,123.209999
2515,1614263400,124.349998,122.339996,5687100,123.370003,122.470001


In [13]:
basicStats = basic_stats(symbol)
basicStats

STATS <Response [200]>


{'Previous Close': '122.47',
 'Open': '122.25',
 'Bid': '118.93 x 1200',
 'Ask': '119.18 x 900',
 "Day's Range": '118.88 - 122.20',
 '52 Week Range': '90.56 - 136.10',
 'Volume': '9,050,990',
 'Avg. Volume': '6,306,290',
 'Market Cap': '106.275B',
 'Beta (5Y Monthly)': '1.25',
 'PE Ratio (TTM)': '19.08',
 'EPS (TTM)': '6.23',
 'Earnings Date': 'Apr 19, 2021',
 'Forward Dividend & Yield': '6.52 (5.32%)',
 'Ex-Dividend Date': 'Feb 09, 2021'}

In [14]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['107.69B',
  '112.17B',
  '108.41B',
  '107.56B',
  '98.49B',
  '118.91B'],
 'Enterprise Value 3': ['160.35B',
  '166.96B',
  '164.08B',
  '165.14B',
  '157.78B',
  '179.58B'],
 'Trailing P/E ': ['19.72', '14.26', '13.79', '11.96', '10.49', '15.57'],
 'Forward P/E 1': ['10.95', '10.75', '9.98', '11.15', '8.31', '9.92'],
 'PEG Ratio (5 yr expected) 1': ['1.25',
  '9.35',
  '7.73',
  '9.69',
  '3.59',
  '22.56'],
 'Price/Sales (ttm)': ['1.47', '1.50', '1.44', '1.41', '1.28', '1.56'],
 'Price/Book (mrq)': ['5.23', '5.29', '5.28', '5.38', '4.73', '6.62'],
 'Enterprise Value/Revenue 3': ['2.18',
  '8.20',
  '9.34',
  '9.11',
  '8.98',
  '8.25'],
 'Enterprise Value/EBITDA 6': ['12.71',
  '50.52',
  '42.81',
  '46.22',
  '82.57',
  '29.95'],
 'Beta (5Y Monthly) ': '1.25',
 '52-Week Change 3': '-8.81%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '136.10',
 '52 Week Low 3': '90.56',
 '50-Day Moving Average 3': '123.07',
 '200-Day Moving Average 3': '12